In [1]:
#pragma cling add_include_path("/home/tom_f/miniconda3/envs/cling/include/")
//#pragma cling add_include_path("/home/tom_f/miniconda3/envs/cling/include/x86_64-linux-gnu/")
#pragma cling add_include_path("/home/tom_f/miniconda3/envs/cling/include/glm/")

In [2]:
#include <iostream>
#include <string>
#include <glm/gtx/string_cast.hpp>

#include "camera.h"
#include "Transform.h"
#include "primitive.h"
#include "sampler.h"
#include "interaction.h"

const float pi = 3.14159265 ; // For portability across platforms
float minNum = std::numeric_limits<float>::max();
float maxNum = std::numeric_limits<float>::max();

vec4 myposition = vec4(5, 7, 0, 1);
mat4 x = glm::transpose(mat4(15, 6, 8, 15,
6, 12, 3, 15,
14, 19, 2, 4,
15, 3, 7, 3));
mat3 testMatrix3 = mat3(1, 3, 1, 3, 3, 4, 4, 7, 5);
mat3 invTestMatrix3 = Transform::InvertMat3(testMatrix3);

In file included from input_line_8:4:
./camera.h:3:21: warning: extra tokens at end of #include directive [-Wextra-tokens]
#include "sampler.h";
                    ^
                    //
./camera.h:4:22: warning: extra tokens at end of #include directive [-Wextra-tokens]
#include "geometry.h";
                     ^
                     //
./camera.h:5:23: warning: extra tokens at end of #include directive [-Wextra-tokens]
#include "Transform.h";
                      ^
                      //
IncrementalExecutor::executeFunction: symbol '_ZN9Transform10InvertMat3EN3glm3matILi3ELi3EfLNS0_9qualifierE0EEE' unresolved while linking function '_GLOBAL__sub_I_cling_module_6'!
You are probably missing the definition of Transform::InvertMat3(glm::mat<3, 3, float, (glm::qualifier)0>)
Maybe you need to load the corresponding shared library?


In [2]:
#include <iostream>
#include <string>
#include <glm/gtx/string_cast.hpp>

#include "camera.h"
#include "primitive.h"
#include "sampler.h"
#include "interaction.h"

const float pi = 3.14159265 ; // For portability across platforms
float minNum = std::numeric_limits<float>::max();
float maxNum = std::numeric_limits<float>::max();
    
vec4 myposition = vec4(5, 7, 0, 1);
mat4 x = glm::transpose(mat4(15, 6, 8, 15,
6, 12, 3, 15,
14, 19, 2, 4,
15, 3, 7, 3));
mat3 testMatrix3 = mat3(1, 3, 1, 3, 3, 4, 4, 7, 5);
mat3 invTestMatrix3 = Transform::InvertMat3(testMatrix3);
mat3 M3invM3 = invTestMatrix3 * testMatrix3;
Transform rotation = Transform::Rotate(45, vec3(0,0,1));
Transform translation = Transform::Translate(vec3(-1.5, -0.8, 0.65));
Transform scaling = Transform::Scale(vec3(0.4));
mat4 compoundMatrix = rotation.m * translation.m * scaling.m;
Transform myTransform = Transform(compoundMatrix);
Transform translate2 = Transform::Translate(vec3(1, -3, -2));

// Ray tests
vec3 originPoint = vec3(0,0,0);
vec3 direction = vec3(0,1,0);
Ray myRay = Ray(originPoint, direction);
vec3 destinationPoint = myRay(10);

// Sphere tests
Sphere mySphere(1, myTransform);
SurfaceInteraction *sphereSurface = new SurfaceInteraction();
Bounds objBounds = mySphere.ObjectBound();
Bounds wldBounds = mySphere.WorldBound();

float t0, t1;

bool doesIntersect = wldBounds.Intersect(myRay, &t0, &t1);
std::cout << std::to_string(t0) << std::endl;
std::cout << std::to_string(t1) << std::endl;

doesIntersect = mySphere.Intersect(myRay, &t0, sphereSurface);
std::cout << std::to_string(t0) << std::endl;
std::cout << glm::to_string(sphereSurface->localGeo.point) << std::endl;
std::cout << glm::to_string(sphereSurface->localGeo.normal) << std::endl;

// Ellipse tests
Transform ellipseTranslate = Transform::Translate(vec3(2, 1.5, -1.5));
Transform ellipseRotate = Transform::Rotate(45, vec3(0, 0, 1));
Transform ellipseScale = Transform::Scale(vec3(1, 1.5, 1));
mat4 ellipseCompoundMatrix = ellipseTranslate.m * ellipseRotate.m * ellipseScale.m;
Transform ellipseCompoundTransform = Transform(ellipseCompoundMatrix);

Ellipsoid myEllipse = Ellipsoid(1, ellipseCompoundMatrix);
SurfaceInteraction* ellipseSurface = new SurfaceInteraction();
Bounds ellipseObjBounds = myEllipse.ObjectBound();
Bounds ellipseWrldBounds = myEllipse.WorldBound();
Ray ellipseRay = Ray(vec3(0, 0, 0), vec3(4, 2.5, -3));

doesIntersect = ellipseWrldBounds.Intersect(ellipseRay, &t0, &t1);
doesIntersect = myEllipse.Intersect(ellipseRay, &t0, ellipseSurface);
vec3 intersectionPoint = ellipseRay(t0);
vec3 ellipseNormal = myEllipse.getNormal(intersectionPoint);
std::cout << std::to_string(t0) << std::endl;
std::cout << std::to_string(t1) << std::endl;
std::cout << glm::to_string(ellipseSurface->localGeo.point) << std::endl;
std::cout << glm::to_string(ellipseSurface->localGeo.normal) << std::endl;

// Triangle tests
vec3 v1, v2, v3;
v1 = vec3(-1, -1, -1);
v2 = vec3(1, -1, -1);
v3 = vec3(-1, -1, 1);

Transform triangleTranslate = Transform::Translate(vec3(0, 4, -4));
Transform triangleScale = Transform::Scale(vec3(2, 1, 0.25));
mat4 triangleCompoundMatrix = triangleTranslate.m * triangleScale.m;
Transform triangleTransform = Transform(triangleCompoundMatrix);

Triangle myTriangle(v1, v2, v3, triangleTransform);
SurfaceInteraction* triangleSurface = new SurfaceInteraction();
Bounds triangleObjBounds = myTriangle.ObjectBound();
Bounds triangleWldBounds = myTriangle.WorldBound();

Ray triangleRay = Ray(vec3(0, 0, 0), vec3(0, 3.6, -4.9));
doesIntersect = triangleWldBounds.Intersect(triangleRay, &t0, &t1);
doesIntersect = myTriangle.Intersect(triangleRay, &t0, triangleSurface);
vec3 triangleIntersectionPoint = triangleRay(t0);
vec3 triangleNormal = myTriangle.getNormal();

// Camera tests
Camera myCamera = Camera(640, 480, 45, vec3(0, -4, 4), vec3(0, -1, 0), vec3(0, 1, 1));
Ray myCameraRay = Ray();
std::cout << glm::to_string(myCameraRay.dir) << std::endl;
Sample mySample = Sample{ 0, 240 };
myCamera.generateRay(mySample, &myCameraRay);
std::cout << glm::to_string(myCameraRay.dir) << std::endl;
// Material tests
Material diffuseMaterial = Material(new BRDF{ 1,0,0,0,Color(1,0,0), 
Color(0,0,0), Color(0,0,0), Color(0,0,0) });
Material specularMaterial = Material(new BRDF{ 0,1,0,0,Color(0,0,0), 
Color(0,1,0), Color(0,0,0), Color(0,0,0) });
Material ambientMaterial = Material(new BRDF{ 0,0,1,0,Color(0,0,0), 
Color(0,0,0), Color(0,0,1), Color(0,0,0) });

// Primitive tests
std::shared_ptr<Sphere> pMySphere = std::make_shared<Sphere>(mySphere);
std::shared_ptr<Material> pDiffuseMaterial = std::make_shared<Material>(diffuseMaterial);
GeometricPrimitive spherePrimitive = GeometricPrimitive(pMySphere, pDiffuseMaterial);
std::shared_ptr<Ellipsoid> pMyEllipse = std::make_shared<Ellipsoid>(myEllipse);
std::shared_ptr<Material> pSpecularMaterial = std::make_shared<Material>(specularMaterial);
GeometricPrimitive ellipsePrimitive = GeometricPrimitive(pMyEllipse, pSpecularMaterial);
std::shared_ptr<Triangle> pMyTriangle = std::make_shared<Triangle>(myTriangle);
std::shared_ptr<Material> pAmbientMaterial = std::make_shared<Material>(ambientMaterial);
GeometricPrimitive trianglePrimitive = GeometricPrimitive(pMyTriangle, pAmbientMaterial);

AggregatePrimitive shapesPrimitive = AggregatePrimitive();
shapesPrimitive.addPrimitive <GeometricPrimitive> (&spherePrimitive);
shapesPrimitive.addPrimitive <GeometricPrimitive> (&ellipsePrimitive);
shapesPrimitive.addPrimitive <GeometricPrimitive> (&trianglePrimitive);
Bounds shapesWorldBound = shapesPrimitive.WorldBound();
std::cout << glm::to_string(shapesWorldBound.pMin) << std::endl;
std::cout << glm::to_string(shapesWorldBound.pMax) << std::endl;

SurfaceInteraction* shapesPrimitiveSurface = new SurfaceInteraction();
doesIntersect = shapesPrimitive.Intersect(ellipseRay, shapesPrimitiveSurface);
intersectionPoint = ellipseRay(t0);

std::cout << glm::to_string(intersectionPoint) << std::endl;

In file included from input_line_8:4:
./camera.h:3:21: warning: extra tokens at end of #include directive [-Wextra-tokens]
#include "sampler.h";
                    ^
                    //
./camera.h:4:22: warning: extra tokens at end of #include directive [-Wextra-tokens]
#include "geometry.h";
                     ^
                     //
./camera.h:5:23: warning: extra tokens at end of #include directive [-Wextra-tokens]
#include "Transform.h";
                      ^
                      //
IncrementalExecutor::executeFunction: symbol '_ZN6Camera11generateRayER6SampleP3Ray' unresolved while linking function '_GLOBAL__sub_I_cling_module_6'!
You are probably missing the definition of Camera::generateRay(Sample&, Ray*)
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN3RayC1Ev' unresolved while linking function '_GLOBAL__sub_I_cling_module_6'!
You are probably missing the definition of Ray::Ray()
Maybe you need to load the cor

vec3(0.000000, 0.000000, -3327.246094)
